# Build a Dashboard Application with Plotly Dash

<p> In this lab, we will be building a Plotly Dash application for users to perform interactive visual analytics on SpaceX launch data in real-time.</p>

This dashboard application contains input components such as a dropdown list and a range slider to interact with a pie chart and a scatter point chart. We will be guided to build this dashboard application via the following tasks:

 - TASK 1: Add a Launch Site Drop-down Input Component
 - TASK 2: Add a callback function to render success-pie-chart based on selected site dropdown
 - TASK 3: Add a Range Slider to Select Payload
 - TASK 4: Add a callback function to render the success-payload-scatter-chart scatter plot

After visual analysis using the dashboard, we should be able to obtain some insights to answer the following five questions:

1. Which site has the largest successful launches?
2. Which site has the highest launch success rate?
3. Which payload range(s) has the highest launch success rate?
4. Which payload range(s) has the lowest launch success rate?
5. Which F9 Booster version (v1.0, v1.1, FT, B4, B5, etc.) has the highest launch success rate?


In [1]:
# Import required libraries
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px
import wget

In [14]:
#Getting the dataset
spacex_dataset = wget.download("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
# spacex_dash_app = wget.download("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py")


In [10]:
df = pd.read_csv(spacex_dataset)

In [12]:
df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [ ]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)
server = app.server

# Create an app layout

app.layout = html.Div(children=[html.H1('SpaceX Rocket Launch Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                html.P("Designed by Friday James", style={'textAlign': 'center'}),
                                
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown',
                                                options=[
                                                    {'label': 'All Sites', 'value': 'ALL'},
                                                    {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                    {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                    {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                    {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                                ],
                                                value='ALL',
                                                placeholder="Select a Launch Site",
                                                searchable=True
                                                ),

                                html.Br(),

                                # Task 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected,
                                # show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                # Add payload mass slider text
                                html.P(id="slider-text"),
                                
                                # TASK 3: Add a slider to select payload range
                                # dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0: '0', 2500: '2500', 5000: '5000', 7500: '7500',
                                                       10000: '10000'},
                                                value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart'))
                                ])


# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(filtered_df, values='class', names='Launch Site', title='Total Successful Launches By Site')
        return fig
    else:
        # return the outcomes piechart for a selected site
        site_chosen = entered_site
        mask = filtered_df['Launch Site'] == site_chosen
        fig = px.pie(filtered_df[mask], names='class',
                     title=f'Total Successful Launches For Site {site_chosen}')
        return fig


# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'),
              [Input(component_id='payload-slider', component_property='value')])
def get_scatter_chart(entered_site, mass):

    # filter masses from payload slider
    mass_1 = spacex_df['Payload Mass (kg)'] >= float(mass[0])
    mass_2 = spacex_df['Payload Mass (kg)'] <= float(mass[1])
    
    filtered_df = spacex_df[mass_1][mass_2]
    
    if entered_site == 'ALL':

        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color="Booster Version Category",
                         title=f'Correlation between Payload Mass and Launch Success for All Sites for Payload Mass(kg) Between {mass[0]} and {mass[1]}')
        return fig
    else:
        
        # return the outcomes scatter chart for a selected site
        site_chosen = entered_site
        mask = filtered_df['Launch Site'] == site_chosen
        fig = px.scatter(filtered_df[mask], x='Payload Mass (kg)', y='class', color="Booster Version Category",
                         title=f'Correlation between Payload Mass and Launch Success for Site {site_chosen}')
        return fig
    
    
#function to return payload_mass success_rate
@app.callback(Output('slider-text', 'children'),
              [Input(component_id='payload-slider', component_property='value')])
def get_success_rate(mass):

    # filter masses from payload slider
    mass_1 = spacex_df['Payload Mass (kg)'] >= float(mass[0])
    mass_2 = spacex_df['Payload Mass (kg)'] <= float(mass[1])
    
    filtered_df = spacex_df[mass_1][mass_2]
    
    # find success rate
    rate = (filtered_df['class'].value_counts().loc[1])/filtered_df['class'].value_counts().sum() * 100
    rate = 'Payload range (Kg): ' + str(round(rate, 2)) + '% Success Rate'
    
    return rate
    
    
# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Oct/2022 22:03:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2022 22:03:44] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2022 22:03:44] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2022 22:03:44] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2022 22:03:44] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2022 22:03:44] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2022 22:03:44] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2022 22:03:44] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2022 22:03:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2022 22:03:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2022 22:09:18] "POST /_dash-

127.0.0.1 - - [15/Oct/2022 22:16:13] "POST /_dash-update-component HTTP/1.1" 200 -
<ipython-input-15-2fe0d1602252>:112: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

127.0.0.1 - - [15/Oct/2022 22:16:18] "POST /_dash-update-component HTTP/1.1" 200 -
<ipython-input-15-2fe0d1602252>:86: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

127.0.0.1 - - [15/Oct/2022 22:16:18] "POST /_dash-update-component HTTP/1.1" 200 -
<ipython-input-15-2fe0d1602252>:112: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

127.0.0.1 - - [15/Oct/2022 22:16:21] "POST /_dash-update-component HTTP/1.1" 200 -
<ipython-input-15-2fe0d1602252>:86: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

127.0.0.1 - - [15/Oct/2022 22:16:21] "POST /_dash-update-component HTTP/1.1" 200 -
<ipython-input-15-2fe0d1602252>:112: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

127.0.0.1 - 

## Author

<a href="https://www.linkedin.com/in/friday-james">Friday James</a> is a Computer Science PhD Candidate in Kansas State University. His research is focused on using machine learning, data science and computer vision to improve teaching and learning in CS Education. Specifically, tracking and forecasting student performance to identify and grow early struggling students in a class. 
